# Applying Neural Networks to NBA Team Win Prediction

In [1]:
# import packages
import os
import numpy as np
import pandas as pd

### Data Cleaning and Aggregation

In [2]:
# create a list to hold all dataset names
dataset_list = []

# adds all CSV files from the datasets folder to a list
for dataset in os.listdir('./win_predictor_data/'):
    if (dataset.endswith('.csv')):
        dataset_list.append(dataset)

# adds all datasets to a dictionary with the key being 'Year Type' 
datasets = {}
for dataset in dataset_list:
    data_file = pd.read_csv('./win_predictor_data/' + dataset)
    dataset_name = dataset[dataset.index('- ') + 1 : dataset.index('.')].strip()
    print(dataset_name)
    datasets[dataset_name] = data_file

2018-2019 Team
2018-2019 Standings
2017-2018 Standings
2016-2017 Misc
2016-2017 Standings
2015-2016 Misc
2017-2018 Misc
2015-2016 Team
2015-2016 Standings
2017-2018 Team
2016-2017 Team
2018-2019 Misc


In [15]:
def remove_asterisk(team):
    if team.endswith("*"):
        return team[0:len(team)-1]
    return team

In [18]:
agg_datasets = {}



for dataset in sorted(datasets):
    
    year_label = dataset[0 : dataset.index(' ')]
    datasets[dataset]['Team'] = datasets[dataset]['Team'].apply(remove_asterisk)
    if year_label in agg_datasets:
        agg_datasets[year_label] = pd.merge(agg_datasets[year_label], datasets[dataset], on='Team')
    else:
        agg_datasets[year_label] = datasets[dataset]
        
agg_datasets['2015-2016']

,Rk_x,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Golden State Warriors,27.4,73.0,9.0,65,17,10.76,-0.38,10.38,...,0.763,10.0,36.2,46.2,28.9,8.4,6.1,15.2,20.7,114.9
1,2.0,San Antonio Spurs,30.3,67.0,15.0,67,15,10.63,-0.36,10.28,...,0.803,9.4,34.5,43.9,24.5,8.3,5.9,13.1,17.5,103.5
2,3.0,Oklahoma City Thunder,25.8,55.0,27.0,59,23,7.28,-0.19,7.09,...,0.782,13.1,35.6,48.6,23.0,7.4,5.9,15.9,20.6,110.2
3,4.0,Cleveland Cavaliers,28.1,57.0,25.0,57,25,6.00,-0.55,5.45,...,0.748,10.6,33.9,44.5,22.7,6.7,3.9,13.6,20.3,104.3
4,5.0,Los Angeles Clippers,29.7,53.0,29.0,53,29,4.28,-0.15,4.13,...,0.692,8.8,33.3,42.0,22.8,8.6,5.6,13.0,21.3,104.5
5,6.0,Toronto Raptors,26.3,56.0,26.0,53,29,4.50,-0.42,4.08,...,0.777,10.2,33.2,43.4,18.7,7.8,5.5,13.1,19.6,102.7
6,7.0,Atlanta Hawks,28.2,48.0,34.0,51,31,3.61,-0.12,3.49,...,0.783,8.3,33.8,42.1,25.6,9.1,5.9,15.0,19.1,102.8
7,8.0,Boston Celtics,25.2,48.0,34.0,50,32,3.21,-0.37,2.84,...,0.788,11.6,33.3,44.9,24.2,9.2,4.2,13.7,21.9,105.7
8,9.0,Charlotte Hornets,26.0,48.0,34.0,49,33,2.72,-0.36,2.36,...,0.790,9.0,35.0,43.9,21.7,7.3,5.3,12.6,18.1,103.4
9,10.0,Utah Jazz,24.2,40.0,42.0,46,36,1.79,0.05,1.84,...,0.744,10.7,32.5,43.2,19.0,7.7,5.2,14.9,20.2,97.7


In [ ]:







    
    np_datasets =  {} ##this will be a dictionary of arrays

    for key in sorted (datasets.keys()): 
        if key[5:len(key)] in stat_types:
            season_data = datasets[key]
            
            if key[0:4] in np_datasets:
                np_datasets[key[0:4]] = pd.merge(np_datasets[key[0:4]], season_data, on = ['Team'])
            else:
                np_datasets[key[0:4]] = season_data

    ## Adds the year to team name to provide unique identification to each team      
    team_data = {}
    for key in np_datasets:

            for index in np_datasets[key].index:
                if 'League Average' not in np_datasets[key].loc[index, 'Team']:
                    team_key = np_datasets[key].loc[index, 'Team'] + " " + key
                    team_data[team_key] = np_datasets[key].loc[index,:]


    agg_data = pd.DataFrame.from_dict(team_data, orient='index')
    agg_data = agg_data.loc[:, stats]
    
    
    return agg_data